<a href="https://colab.research.google.com/github/rahiakela/genai-research-and-practice/blob/main/campusx-langchain/01_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

**Reference**

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/

In [1]:
%%capture

%pip install -qU langchain-google-genai
%pip install langchain langchain-community

In [32]:
import os
import json

from google.colab import userdata

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pydantic import BaseModel, Field

import matplotlib.pyplot as plt
from IPython.display import Image, display
%matplotlib inline

In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=userdata.get('GOOGLE_API_KEY')
)

## Step #1: Naive Approach

In [5]:
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

In [6]:
prompt1 = template1.invoke({"topic":"black hole"})
result = model.invoke(prompt1)

prompt2 = template2.invoke({"text":result.content})
result2 = model.invoke(prompt2)

In [7]:
print(result2.content)

Black holes are regions of extreme gravity where nothing, not even light, can escape. They form primarily from the collapse of massive stars or, hypothetically, directly in the early universe.  Key features include a singularity, an event horizon, and often an accretion disk. Black holes are classified by mass, ranging from stellar-mass to supermassive, and their presence is detected through gravitational lensing, X-ray emissions, stellar orbits, and gravitational waves. They significantly impact galaxy evolution, provide a testing ground for general relativity, and are potential dark matter candidates.  Ongoing research aims to understand their formation, the nature of singularities, and their role in the cosmos.


##Step #2: String Output Parser

In [9]:
parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({"topic":"black hole"})

In [11]:
print(result)

Black holes are regions of extreme gravity where nothing escapes, formed primarily from stellar collapse or direct gas cloud collapse. They possess a singularity, an event horizon, and potentially an ergosphere and accretion disk. Classified by mass (stellar, intermediate, supermassive, primordial), their presence is detected through gravitational lensing, X-ray emissions, stellar orbits, and gravitational waves. Black holes influence galaxy evolution, power active galactic nuclei, and offer a testing ground for general relativity. Unresolved questions remain about their formation, singularities, and role in galaxy evolution, driving future research in observational techniques and theoretical modeling.


##Step #3: Structured Output Parser

In [14]:
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [15]:
chain = template | model | parser

result = chain.invoke({"topic":"black hole"})

In [17]:
print(result["fact_1"])

Black holes are not cosmic vacuum cleaners. While they have immense gravity, you would need to get relatively close to one to be pulled in. If our Sun were replaced with a black hole of the same mass, the Earth would continue orbiting at the same distance, though it would get very cold!


In [18]:
print(result["fact_2"])

Black holes come in different sizes. Stellar mass black holes form from the collapse of massive stars and are typically a few to tens of times the mass of our Sun. Supermassive black holes reside at the centers of most galaxies and can be millions or even billions of times the mass of the Sun.


In [19]:
print(result["fact_3"])

Nothing, not even light, can escape a black hole's event horizon. This boundary marks the point of no return; once something crosses it, it's trapped forever. This is why black holes appear 'black' – no light can reflect off of them or escape their gravity.


##Step #4: Pydantic Output Parser

In [24]:
class Person(BaseModel):
    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

In [26]:
template = PromptTemplate(
    template="Generate the name, age and city of a fictional {place} person \n {format_instruction}",
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser
final_result = chain.invoke({'place':'sri lankan'})

In [27]:
print(final_result)

name='Priya Silva' age=32 city='Colombo'


##Step #5: JSON Output Parser

In [29]:
parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

In [30]:
result = chain.invoke({'topic':'black hole'})

In [31]:
print(result)

{'black_hole_facts': [{'fact': 'Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape.'}, {'fact': "The boundary beyond which escape is impossible is called the event horizon. It's not a physical surface, but a point of no return."}, {'fact': 'Black holes are formed from the remnants of massive stars that collapse under their own gravity at the end of their life cycle. Supermassive black holes, millions or billions of times the mass of the Sun, reside at the centers of most galaxies.'}, {'fact': "While we can't directly 'see' a black hole, we can detect them by observing their effects on surrounding matter, such as the accretion disk of superheated gas that forms around them, or the gravitational lensing of light from objects behind them."}, {'fact': "According to Einstein's theory of general relativity, at the center of a black hole is a singularity, a point of infinite density where the laws of physics as we know them break down."}]}

In [35]:
print(json.dumps(result, indent=2))

{
  "black_hole_facts": [
    {
      "fact": "Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape."
    },
    {
      "fact": "The boundary beyond which escape is impossible is called the event horizon. It's not a physical surface, but a point of no return."
    },
    {
      "fact": "Black holes are formed from the remnants of massive stars that collapse under their own gravity at the end of their life cycle. Supermassive black holes, millions or billions of times the mass of the Sun, reside at the centers of most galaxies."
    },
    {
      "fact": "While we can't directly 'see' a black hole, we can detect them by observing their effects on surrounding matter, such as the accretion disk of superheated gas that forms around them, or the gravitational lensing of light from objects behind them."
    },
    {
      "fact": "According to Einstein's theory of general relativity, at the center of a black hole is a singularity, a poin